In [1]:
!pip install pyrosetta-installer 
!python -c 'import pyrosetta_installer; pyrosetta_installer.install_pyrosetta()'

  Using cached pyrosetta_installer-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached pyrosetta_installer-0.1.2-py3-none-any.whl (3.9 kB)
PyRosetta install detected, doing nothing...


In [3]:
from pyrosetta import *

# If you need to relax multiple structures, it's better to run `init()` each time
init()

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.conda.ubuntu.cxx11thread.serialization.Ubuntu.python313.Release 2025.06+release.029c6a159b896477003a14f78f472d4cd2cead46 2025-02-04T15:14:13] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.conda.ubuntu.cxx11thread.ser

In [ ]:
# Change the working directory to where the PDB file is located
pose = pose_from_pdb("prepared_structures/4I2Y.pdb")

# copy pose to a new Pose object called testPose
testPose = Pose()
testPose.assign(pose)
print(testPose)

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.600926 seconds.
core.import_pose.import_pose: File '4I2Y.pdb' automatically determined to be of type PDB
core.chemical.GlobalResidueTypeSet: Loading (but possibly not actually using) 'NRQ' from the PDB components dictionary for residue type 'pdb_NRQ'
core.conformation.Conformation: [ WARNING ] missing heavyatom: OXT on residue pdb_NRQ 159
core.chemical.AtomICoor: [ WARNING ] IcoorAtomID::atom_id(): Cannot get atom_id for POLYMER_LOWER of residue SER 160.  Returning BOGUS ID instead.
core.conformation.Residue: [ WARNING ] missing an atom: 160  H   that depends on a nonexistent polymer connection!
core.conformation.Residue: [ WARNING ]  --> generating it using idealized coordinates.
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue MET:CtermProteinFull 382
core.conformation.Conformati

### **Step 1. Prepare the starting structure with `FastRelax()`**

- It's better not to run `FastRelax()` on Colab, sometimes it will crush the kernel. 
- This step will take a really long time, typically 3-5 hours, it depends on the length of your protein sequences. But luckly, you only need to run `FastRelax()` once for your protein. Save the output relaxed PDB structure, then you can move to STEP2.

In [ ]:
from pyrosetta.rosetta.protocols.relax import FastRelax

# Create a score function
scorefxn = get_fa_scorefxn()

# Set up FastRelax
relax = FastRelax()
relax.set_scorefxn(scorefxn)
relax.constrain_relax_to_start_coords(True)

relax.apply(testPose)
# Change the name of the output file
testPose.dump_pdb("4I2Y_relaxed.pdb")

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
protocols.relax.RelaxScriptManager: Reading relax scripts list from database.
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
protocols.relax.RelaxScriptManager: Looking for MonomerRelax2019.txt
protocols.relax.RelaxScriptManager: ================== Reading script file: /home/shuyu/anaconda3/envs/Rosetta/lib/python3.11/site-packages/pyrosetta/database/sampling/relax_scripts/MonomerRelax2019.txt ==================
protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
protocols.relax.RelaxScriptManager: coord_cst_weight 1.0
protocols.relax.RelaxScriptManager: scale:fa_rep 0.040
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.051
protocols.relax.RelaxScriptManager: min 0.01
protocols.relax.RelaxScriptManager: coord_cst_weight 0.5
protocols.relax.RelaxScriptManager: scale:fa_rep 0.265
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 

True

#### **Extract single chain from relaxed PDB file**

Only do this step when your PDB file has several same chains.

In [4]:
def extract_chain(pdb_file, chain_id, output_file):
     with open(pdb_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            if line.startswith("ATOM") and line[21] == chain_id:
                outfile.write(line)
            elif (line.startswith("TER") and line[21] == chain_id) or line.startswith("END"):
                outfile.write(line)

extract_chain("4I2Y_relaxed.pdb", "A", "4I2Y_relaxed_chain_A.pdb")

**Note to myself: After relaxed, align 4I2Y.pdb and 4I2Y_relaxed_chain_A.pdb in ChimeraX.** 

ChimeraX Log: <br>
Matchmaker 4I2Y.pdb, chain A (#1) with 4I2Y_relaxed.pdb, chain A (#2), sequence alignment score = 1953.3
RMSD between 377 pruned atom pairs is 0.839 angstroms; (across all 381 pairs: 0.862)
